In [227]:
import pandas as pd
import json
from module.load_json import *
from module.utils import *
from module.maps import *

In [228]:
train_path = 'dataset/NIKL_ABSA_2021_v1.0/EXSA2112203180.json'
dev_path = 'dataset/NIKL_ABSA_2021_v1.0/EXSA2122203180.json'

In [229]:
train = jsonload(train_path)
dev = jsonload(dev_path)
train = pd.DataFrame(train['document'])
dev = pd.DataFrame(dev['document'])
total = pd.concat([train, dev]).reset_index(drop=True)

In [230]:
total.domain.unique()
categories = ['제품 기타', '전자기기', '화장품/세정제']

checker = total.domain.apply(lambda x: x in categories)
total = total[checker]
total = total[total.isna().any(axis=1) == False]
total = total[['domain', 'sentence', 'opinions']].reset_index(drop=True)

In [231]:
def collect_sentences(sentences):
    collected_sentences = []
    for sentence in sentences:
        collected_sentences.append(sentence['sentence_form'])
    return ' '.join(collected_sentences)

In [232]:
def collect_opinions(opinions):
    collected_opinions = []
    for opinion in opinions:
        category = opinion['category']
        polarity = opinion['opinion polarity']
        if polarity != 'conflict':
            collected_opinions.append([category, [], polarity])
    return collected_opinions

In [233]:
total['sentence'] = total.sentence.apply(collect_sentences)
total['opinions'] = total.opinions.apply(collect_opinions)
total = total[total.opinions.apply(lambda x: x == []) == False]
total = total.reset_index(drop=True)
total = total.set_axis(['id', 'sentence_form', 'annotation'], axis='columns')

In [234]:
count_tags(total, entity_property_pair)

tags found:  1195
tag set of df:  20
tag set of offered:  25
difference:  {'제품 전체#다양성', '패키지/구성품#가격', '본품#인지도', '브랜드#디자인', '본품#가격'}
제품 전체#일반		290
본품#품질		178
본품#일반		148
제품 전체#품질		103
제품 전체#디자인	75
본품#편의성		58
제품 전체#인지도	58
제품 전체#편의성	56
브랜드#일반		48
패키지/구성품#디자인	43
제품 전체#가격		33
패키지/구성품#편의성	33
패키지/구성품#일반	19
본품#다양성		19
본품#디자인		9
패키지/구성품#품질	7
브랜드#품질		7
브랜드#인지도		7
브랜드#가격		3
패키지/구성품#다양성	1


In [235]:
DATA_V = 'uncleaned_v23'
save_path = f'./dataset/{DATA_V}'
print(save_path)

./dataset/uncleaned_v23


In [237]:
!mkdir -p {save_path}

total.to_csv(f'{save_path}/absa2021.csv', index=False, encoding='utf-8-sig')